In [1]:
# BENEX

import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

#* BUERABIBX
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn import neighbors

# MBER

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, classification_report

# DRBAX
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # 指定使用的GPU编号
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at:', device_name)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Found GPU at: /device:GPU:0


In [2]:
initial_data = pd.read_csv('K:\\Data\\MachineLearningCSV\\MachineLearningCVE\\total4.csv')

In [3]:
initial_data.head(n=5)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [4]:
initial_data.shape

(2827876, 78)

In [5]:
data_to_use = initial_data.dropna()
data_to_use.shape

(2827876, 78)

In [6]:
data_to_use.head(n=5)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [7]:
X = data_to_use.drop(axis=1, columns=[ 'Label'])
#X = X.drop(axis=1, columns=['label'])

y1 = data_to_use['Label'].values
#y2 = data_to_use[ 'label'].values

# SMOTE上采样
#smote = SMOTE()
#X , y1 = smote.fit_resample(X, y1)
# ADASYN上采样
#adasyn = ADASYN()
#X, y1 = adasyn.fit_resample(X, y1)
"""
# 创建权重字典
class_weights = {"BENIGN":                       1.0,
                "DoS Hulk":                      5,
                "PortScan":                      7,
                "DDoS":                          7,
                "DoS GoldenEye":                 10,
                "FTP-Patator":                   10,
                "SSH-Patator":                   10,
                "DoS slowloris":                 10,
                "DoS Slowhttptest":              10,
                "Bot":                           12,
                "Web Attack � Brute Force":      12,
                "Web Attack � XSS":              15,
                "Infiltration":                  20,
                "Web Attack � Sql Injection":    50,
                "Heartbleed":                    50,
                }

# 根据每个样本的标签在权重字典中查找相应的权重值
sample_weights = np.array([class_weights[label] for label in y1])
"""

# 划分训练集和测试集
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=123)

In [8]:
def data_ratio(y1):
    unique, count = np.unique(y1, return_counts=True)
    ratio = round(count[0]/count[1], 1)
    return f'{ratio}:1 ({count[0]}/{count[1]})'

In [9]:
#print(X[:3])

In [10]:
#fig= plt.subplots(figsize=(13,5))
#sns.countplot(x=data_to_use['attack_cat'])
#fig= plt.subplots(figsize=(7,5))
#sns.countplot(x=data_to_use['label'])

In [11]:
"""
#test_data = pd.read_csv('K:\\Data\\MachineLearningCSV\\MachineLearningCVE\\total4.csv')
X_test = test_data.drop(axis=1, columns=['Label'])
#X_test = X_test.drop(axis=1, columns=['label'])

y1_test = test_data['Label'].values
#y2_test = test_data['label'].values
X_train = X

y1_train = y1
#y2_train = y2
"""

"\n#test_data = pd.read_csv('K:\\Data\\MachineLearningCSV\\MachineLearningCVE\\total4.csv')\nX_test = test_data.drop(axis=1, columns=['Label'])\n#X_test = X_test.drop(axis=1, columns=['label'])\n\ny1_test = test_data['Label'].values\n#y2_test = test_data['label'].values\nX_train = X\n\ny1_train = y1\n#y2_train = y2\n"

In [12]:
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'bool']).columns

In [13]:
numerical_cols

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Co

In [14]:
categorical_cols

Index([], dtype='object')

In [15]:
import joblib
t = [('ohe', OneHotEncoder(drop='first'), categorical_cols),
('scale', StandardScaler(), numerical_cols)]

col_trans = ColumnTransformer(transformers=t)
col_trans.fit(X_train)
joblib.dump(col_trans, 'col_trans.joblib')

col_trans = joblib.load('col_trans.joblib')

In [16]:
X_train_transform = col_trans.transform(X_train)

In [17]:
X_test_transform = col_trans.transform(X_test)

In [18]:
X_train_transform.shape

(2262300, 77)

In [19]:
X_test_transform.shape

(565576, 77)

In [20]:
pd.unique(y1)

array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'FTP-Patator', 'SSH-Patator',
       'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye',
       'Heartbleed'], dtype=object)

In [21]:
target_trans = LabelEncoder()
#y_encoded = target_trans.fit(y1_train)
y_encoded = target_trans.fit_transform(y1_train)
print(pd.unique(y_encoded))

[ 0  2 10  6  4  3  7 12 11  5  1 14 13  8  9]


In [22]:
y1_train_transform = target_trans.transform(y1_train)
y1_test_transform = target_trans.transform(y1_test)

In [23]:
#pd.set_option('display.max_rows', None)
#print(X_train_transform[:1])

[[-0.43839652 -0.4390986  -0.01117066 -0.00942907 -0.04772149 -0.00706403
  -0.22700992  0.43548744 -0.070903   -0.24537129 -0.39163784  0.98606295
  -0.32557127 -0.39948605 -0.0572983  -0.27813922 -0.28292716 -0.36322122
  -0.37483487 -0.04713318 -0.43181105 -0.27427096 -0.33922355 -0.36905252
  -0.11918756 -0.34456991 -0.20333834 -0.23670881 -0.27319976 -0.11654086
  -0.22067228  0.         -0.01042836  0.          0.00128657  0.00161874
  -0.25795971 -0.18236124  1.12841953 -0.41502167 -0.34588503 -0.40851428
  -0.29421712 -0.19121231 -0.22067228 -0.015693   -0.65234568 -0.6786085
  -0.32367406 -0.01042836 -0.01572116  0.46228777 -0.27884938 -0.070903
  -0.32557127  0.          0.          0.          0.          0.
   0.         -0.01117066 -0.04779243 -0.00942907 -0.00706384 -0.48783261
  -0.23562684 -0.00844654  0.0026115  -0.12778382 -0.10461577 -0.15014282
  -0.10310073 -0.35226162 -0.10964624 -0.35721419 -0.3393042 ]]


In [24]:
#print(DataFrame(X_train_transform[:3]))

         0         1         2         3         4         5         6   \
0 -0.438397 -0.439099 -0.011171 -0.009429 -0.047721 -0.007064 -0.227010   
1 -0.417058 -0.439791 -0.009827 -0.009429 -0.048478 -0.007112 -0.238178   
2 -0.436919  2.192229 -0.000424 -0.006399 -0.046115 -0.001975 -0.261910   

         7         8         9   ...        67        68        69        70  \
0  0.435487 -0.070903 -0.245371  ... -0.008447  0.002611 -0.127784 -0.104616   
1 -0.310271 -0.213611 -0.152160  ... -0.008447  0.002622 -0.127784 -0.104616   
2 -0.310271 -0.276140 -0.226482  ...  0.002256  0.002611 -0.121484 -0.104616   

         71        72        73         74        75        76  
0 -0.150143 -0.103101 -0.352262  -0.109646 -0.357214 -0.339304  
1 -0.150143 -0.103101 -0.352262  -0.109646 -0.357214 -0.339304  
2 -0.146210 -0.095982  1.508764  11.530974  3.002106 -0.075878  

[3 rows x 77 columns]


In [25]:
#clf = LogisticRegression(solver='lbfgs', random_state=123, max_iter = 4000, multi_class = "ovr")
#clf = DecisionTreeClassifier(random_state=123)
clf = RandomForestClassifier(random_state=123, max_depth=10, min_samples_split=5, min_samples_leaf=2)
#clf = neighbors.KNeighborsClassifier()
#model = MLPClassifier(random_state=123, solver='adam', max_iter=8000)
#cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

#scoring = ['accuracy', 'precision_micro', 'recall_micro', 'f1_micro', 'roc_auc_ovr']

#cv_results = cross_validate(estimator=clf,
#                            X=X_train_transform,
#                            y=y1_train_transform,
 #                           scoring=scoring,
#                            cv=cv,
#                            return_train_score=False)

In [26]:
#MLPmodel = [('MultiLayerPerceptron', MLPClassifier(random_state=123, solver='adam', max_iter=8000))]

In [27]:
#for  model_name,clf in MLPmodel:
#    print(clf)

In [28]:
#cv_results['test_accuracy'].mean()

In [29]:

clf.fit(X=X_train_transform, y=y1_train_transform)

y_pred_class = clf.predict(X=X_test_transform)
y_pred_score = clf.predict_proba(X=X_test_transform)[:, 1]

# IEEE

cm_ontest = confusion_matrix(y_true=y1_test_transform, y_pred=y_pred_class)
# precision HM

precision_ontest = precision_score(y_true=y1_test_transform, y_pred=y_pred_class, average='micro')
# recall DH
recall_ontest = recall_score(y_true=y1_test_transform, y_pred=y_pred_class, average='micro')

# DRE
cls_report_ontest = classification_report(y_true=y1_test_transform, y_pred=y_pred_class)

# FTEIER

print('The precision score on the test set: {:1.5f}'.format(precision_ontest))
print('The recall score on the test set: {:1.5f}'.format(recall_ontest))
print('Confusion Matrix:\n', cm_ontest)

# DERE

print('Classification Report:\n', cls_report_ontest)



K:\Anaconda\envs\mytensor\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The precision score on the test set: 0.99555
The recall score on the test set: 0.99555
Confusion Matrix:
 [[454102      0     10      1    180     45      0      0      0      0
     186      0      0      0      0]
 [   389      5      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [    56      0  25427      0      0      0      0      0      0      0
       0      0      0      0      0]
 [   168      0      0   1876      1      2      0      0      0      0
       0      0      0      0      0]
 [   519      0      1      0  45584      0      0      0      0      0
       0      0      0      0      0]
 [    91      0      0      0      0    977      0      0      0      0
       0      0      0      0      0]
 [    81      0      0      0      0      0   1078      0      0      0
       0      0      0      0      0]
 [     5      0      0      0      0      0      0   1586      0      0
       0      0      0      0      0]
 [     0      

K:\Anaconda\envs\mytensor\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
K:\Anaconda\envs\mytensor\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
#print(clf.feature_importances_)

In [31]:
"""
import pickle

with open("crmodel.pkl", "wb") as f:
    pickle.dump(clf, f)
    
# Load the model from file
with open("crmodel.pkl", "rb") as f:
    loaded_model = pickle.load(f)

y_pred_class = loaded_model.predict(X=X_test_transform)
y_pred_score = loaded_model.predict_proba(X=X_test_transform)

cm_ontest = confusion_matrix(y_true=y1_test_transform, y_pred=y_pred_class)

precision_ontest = precision_score(y_true=y1_test_transform, y_pred=y_pred_class, average='micro')

recall_ontest = recall_score(y_true=y1_test_transform, y_pred=y_pred_class, average='micro')

cls_report_ontest = classification_report(y_true=y1_test_transform, y_pred=y_pred_class)

print('The precision score on the test set: {:1.5f}'.format(precision_ontest))
print('The recall score on the test set: {:1.5f}'.format(recall_ontest))
print('Confusion Matrix:\n', cm_ontest)

print('Classification Report:\n', cls_report_ontest)
"""

'\nimport pickle\n\nwith open("crmodel.pkl", "wb") as f:\n    pickle.dump(clf, f)\n    \n# Load the model from file\nwith open("crmodel.pkl", "rb") as f:\n    loaded_model = pickle.load(f)\n\ny_pred_class = loaded_model.predict(X=X_test_transform)\ny_pred_score = loaded_model.predict_proba(X=X_test_transform)\n\ncm_ontest = confusion_matrix(y_true=y1_test_transform, y_pred=y_pred_class)\n\nprecision_ontest = precision_score(y_true=y1_test_transform, y_pred=y_pred_class, average=\'micro\')\n\nrecall_ontest = recall_score(y_true=y1_test_transform, y_pred=y_pred_class, average=\'micro\')\n\ncls_report_ontest = classification_report(y_true=y1_test_transform, y_pred=y_pred_class)\n\nprint(\'The precision score on the test set: {:1.5f}\'.format(precision_ontest))\nprint(\'The recall score on the test set: {:1.5f}\'.format(recall_ontest))\nprint(\'Confusion Matrix:\n\', cm_ontest)\n\nprint(\'Classification Report:\n\', cls_report_ontest)\n'

In [32]:
#models = [('LogisticRegression',LogisticRegression(random_state=123, max_iter=5000, multi_class = "ovr")),
#        ('DecisionTree', DecisionTreeClassifier(random_state=123)),
#        ('RandomForest', RandomForestClassifier(random_state=123)),
#        ]

In [33]:
#results_dict1 = {'Model Name': model_names_list,
#PETER
#                'CV Fit Time': cv_fit_time_mean_list,
#                'CV Accuracy mean': cv_accuracy_mean_list,
#                'CV Precision mean': cv_precision_mean_list,
#                'CV Recall mean': cv_recall_mean_list,
#                'CV F1 mean': cv_f1_mean_list,
#
#                'Test Accuracy': test_accuracy_list,
#                'Test Precision': test_precision_list,
#                'Test Recall': test_recall_list,
#                'Test F1': test_f1_list,
#                }
#results_df = pd.DataFrame(results_dict)
#
#4 BRUM LE OX NETE
#results_df1.sort_values(by='Test F1', ascending=False)

In [34]:
"""
import pandas as pd
import pickle

# Load the model from file
with open("rmodel.pkl", "rb") as f:
    loaded_model = pickle.load(f)

# Load the data from file
#data = pd.read_csv('data.csv')

# Make predictions on the data
y_pred_class = loaded_model.predict(X=X_test_transform)
y_pred_score = loaded_model.predict_proba(X=X_test_transform)

print(y_pred_class)
"""

'\nimport pandas as pd\nimport pickle\n\n# Load the model from file\nwith open("rmodel.pkl", "rb") as f:\n    loaded_model = pickle.load(f)\n\n# Load the data from file\n#data = pd.read_csv(\'data.csv\')\n\n# Make predictions on the data\ny_pred_class = loaded_model.predict(X=X_test_transform)\ny_pred_score = loaded_model.predict_proba(X=X_test_transform)\n\nprint(y_pred_class)\n'